In [1]:
from numpy import mean
from numpy import std
import numpy as np
import os
import pandas as pd
from numpy import absolute
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR

In [2]:
NUMBER_NODES = 7

def load_data():
    train_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_train.csv'))
    val_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_val.csv'))
    test_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_test.csv'))

    featuresNumber = (NUMBER_NODES * NUMBER_NODES - NUMBER_NODES) // 2 
    def get_tuple_tensor_dataset(row):
        X = row[0 : featuresNumber].astype('float32')
        Y = row[featuresNumber + 1: ].astype('float32') # Inclui a banda otima na posicao 0
        return X, Y

    train_dataset = list(map(get_tuple_tensor_dataset, train_df.to_numpy()))
    val_dataset = list(map(get_tuple_tensor_dataset, val_df.to_numpy()))
    test_dataset = list(map(get_tuple_tensor_dataset, test_df.to_numpy()))

    X = []
    Y = []
    for x, y in train_dataset:
        X.append(x)
        Y.append(y)
    x_train = np.array(X, dtype=object)
    y_train = np.array(Y, dtype=object)

    X = []
    Y = []
    for x, y in test_dataset:
        X.append(x)
        Y.append(y)
    x_test = np.array(X, dtype=object)
    y_test = np.array(Y, dtype=object)


    return x_train, y_train, np.array(val_dataset, dtype=object), x_test, y_test

In [3]:
x, y, val, x_t, y_t = load_data()

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# define base model
# model = DecisionTreeRegressor()
model = DecisionTreeRegressor()
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# define the chained multioutput wrapper model
# wrapper = RegressorChain(model)

model.fit(x, y)

MAE: 1.578 (0.090)


DecisionTreeRegressor()

In [12]:
import networkx as nx

def getGraph(upperTriangleAdjMatrix):
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    k = 0
    for i in range(NUMBER_NODES):
        for j in range(NUMBER_NODES):
            if i == j:
                continue
            elif i < j:
                dense_adj[i][j] = upperTriangleAdjMatrix[k]
                k += 1
            else:
                dense_adj[i][j] = dense_adj[j][i]
    return dense_adj

def get_bandwidth(Graph, nodelist):
    Graph = nx.Graph(Graph)
    if not Graph.edges:
        return 0
    if nodelist:
        L = nx.laplacian_matrix(Graph, nodelist=nodelist)
    else:
        L = nx.laplacian_matrix(Graph)
    x, y = np.nonzero(L)
    return (x-y).max()

In [13]:
x_t.shape

(63, 21)

In [15]:
pred = model.predict(x_t)
# summarize the prediction

sumTest_pred = 0
sumTest_true = 0
sumTest_original = 0
for i in range(len(pred)):
    graph = getGraph(x_t[i])
    
    print("Pred")
    print(pred[i])
    print("True")
    print(y_t[i])
    
    original_band = get_bandwidth(graph, None)
    sumTest_original += original_band
    print(original_band)
    pred_band = get_bandwidth(graph, pred[i])
    sumTest_pred += pred_band
    true_band = get_bandwidth(graph, y_t[i])
    sumTest_true += true_band
    print("Bandwidth")
    print(pred_band)
    print(true_band)

Pred
[5. 1. 6. 0. 4. 2. 3.]
True
[1.0 5.0 6.0 0.0 3.0 2.0 4.0]
6


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
print("Bandwidth mean")
print(sum_original / pred.shape[0])
print("Pred bandwidth mean")
print(sumTest_pred / pred.shape[0])
print("True bandwidth mean")
print(sumTest_true / pred.shape[0])

Pred bandwidth mean
3.6984126984126986
True bandwidth mean
3.1904761904761907


In [ ]:
def count_repeats(output):
    true_shape = NUMBER_NODES
    counts = np.unique(np.round(output))
    repeated = true_shape - counts.shape[0]
    return repeated


In [ ]:
a = pred[0]
a

array([5., 1., 6., 0., 4., 2., 3.])

In [ ]:
count = 0
for i in pred:
    count += count_repeats(i)
print(count)

0


# Notes

## No Chained Multioutput Regression
- Only DecisionTreeRegressor, counted repeats: 0
- Only LinearRegression, counted repeats: 238
- Only KNeighborsRegressor, counted repeats: I couldn't get the output (infinite loading)

## Chained Multioutput Regression

- DecisionTreeRegressor, counted repeats: 26, 30
- LinearRegression, counted repeats: 238
- KNeighborsRegressor, counted repeats: 119
- SVM, actually support vector regression: 213

## No Chained Multioutput Regression but treating eache output independently (wrong)
- SVM, actually support vector regression: 194
- KNeighborsRegressor, counted repeats: I couldn't get the output (infinite loading)
- LinearRegression, counted repeats: 238
- DecisionTreeRegressor, counted repeats: 116

### https://machinelearningmastery.com/multi-output-regression-models-with-python/

#### Essa abordagens eu testei:
https://machinelearningmastery.com/multi-label-classification-with-deep-learning/
(sigmoid no final, e binary_crossentropy como loss)

https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-satellite-photos-of-the-amazon-rainforest/